In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [9]:
import pandas as pd

In [10]:
options = ChromeOptions()
service = ChromeService(executable_path=ChromeDriverManager().install())

In [11]:
browser = webdriver.Chrome(service=service, options=options)

browser.get("https://www.jstage.jst.go.jp/browse/-char/ja")
browser.implicitly_wait(10)

# 영어로 변환
language = browser.find_element(By.XPATH, '/html/body/span[1]/header/nav/div/div[3]/ul/li[4]/a')
language.click()
time.sleep(1)

eng = browser.find_element(By.XPATH, '/html/body/span[1]/header/nav/div/div[3]/ul/li[4]/ul/li[2]/a')
eng.click()
time.sleep(1)

serch = browser.find_element(By.CSS_SELECTOR, 'input.hm-search-input')
serch.click()
time.sleep(2)

serch.send_keys("地域スポーツクラブ")
serch.send_keys(Keys.ENTER)
time.sleep(2)

check_box = browser.find_element(By.CSS_SELECTOR, 'label[for="srySybt1"].checkbox')
check_box.click()
time.sleep(2)



total_title = []
total_date = []
total_journal = []
total_abstract = []

for _ in range(25):
    hiddens_1 = browser.find_elements(By.CSS_SELECTOR, "span.show_abstract")
    hiddens_2 = browser.find_elements(By.CSS_SELECTOR, "div.print-non-disp")[4:-3]

    for i in range(len(hiddens_1)):
        hiddens_1[i].click()
        time.sleep(1)
        hiddens_2[i].click()
        time.sleep(1)
        
    time.sleep(2)   


    search_results_wrap = browser.find_element(By.ID, "search-resultslist-wrap")

    for i in range(len(search_results_wrap.find_elements(By.TAG_NAME, "li"))):
        li_tag = search_results_wrap.find_elements(By.TAG_NAME, "li")[i]
        # 제목
        try:
            title = li_tag.find_element(By.CLASS_NAME, "searchlist-title").text.strip()
        except:
            title = None
        
        total_title.append(title)
        
        # 저널명
        try:
            journal = li_tag.find_element(By.CLASS_NAME, "searchlist-additional-info").find_element(By.TAG_NAME, "a").text.strip()
        except:
            title = None
        
        total_journal.append(journal)

        # 날짜
        try:
            date = li_tag.find_element(By.CLASS_NAME, "searchlist-additional-info")
            date = date.get_attribute("outerHTML")
            soup = BeautifulSoup(date, "html.parser")
            date_text = soup.get_text(separator="/n", strip=True)
            date_lines = date_text.split("/n")
            date = date_lines[2].strip()[-4:]
        except:
            date = None
        
        total_date.append(date) 

        # 초록
        try:
            # abstract = li_tag.find_element(By.TAG_NAME, "inner-content abstract datapanel_opend").text.strip()
            abstract = li_tag.find_element(By.CLASS_NAME, "inner-content.abstract.datapanel_opend")

            # 찾은 요소의 HTML 코드 가져오기
            abstract = abstract.get_attribute("outerHTML")

            # BeautifulSoup을 사용하여 HTML 파싱
            soup = BeautifulSoup(abstract, "html.parser")

            # 텍스트 추출
            abstract = soup.get_text(separator=" ", strip=True)
        except:
            abstract = None
            
        total_abstract.append(abstract) 
        
        
    # 페이징 버튼
    page = browser.find_elements(By.CSS_SELECTOR, 'a.defaultstyle')
    page[-2].click()
    time.sleep(2)

In [12]:
print(len(total_title))
print(len(total_date))
print(len(total_journal))
print(len(total_abstract))

488
488
488
488


In [13]:
df = pd.DataFrame({"Title":total_title, "Publised":total_date, "Journal":total_journal, "Abstract":total_abstract})

In [14]:
df.to_excel("DataFrame_JP.xlsx", index=False)